### Часть 1

In [ ]:
!pip install kaggle
!sudo apt install unzip
!sudo apt-get install p7zip-full

!export KAGGLE_USERNAME='borispanfilov'
!export KAGGLE_KEY='23e1a26705b833184e1835da72c45fe3'

!kaggle competitions download -c avito-context-ad-clicks
!unzip avito-context-ad-clicks.zip
!7z x VisitsStream.tsv.7z

!head -n 1000000 VisitsStream.tsv > VisitsStream_1M.tsv
!cut -f1 VisitsStream_1M.tsv | sort | uniq -c | sort -nr | head -10 | awk '{printf "%8d %s\n", $1, $2}' > top_users.txt
!hdfs dfs -put -f top_users.txt s3a://avito-top-users/top_users.txt

ссылка на выгруженный файл: https://storage.yandexcloud.net/bspanfilov-lsml/top_users.txt

### Часть 2

In [10]:
!7z x AdsInfo.tsv.7z
! sed -i -e '1'd AdsInfo.tsv
! head -n 2 AdsInfo.tsv

In [ ]:
!hdfs dfs -mkdir -p user/input/ads
!hdfs dfs -put AdsInfo.tsv /user/input/ads

In [1]:
%%writefile CategoryPriceCount.py
import sys

def _rewind_stream(stream):
    for _ in stream:
        pass

def mapper():
    for line in sys.stdin:
        parts = line.strip().split('\t')
        if len(parts) >= 5:
            category_id = parts[2]
            price = parts[4]
            if category_id != 'NULL' and price != 'NULL' and price.isdigit():
                print(f"{category_id}\t{price}")
                
def reducer():
    current_category = None
    total = 0

    for line in sys.stdin:
        line = line.strip()
        category, price = line.split('\t')
        price = float(price)

        if current_category == category:
            total += price
        else:
            if current_category is not None:
                print(f"{current_category}\t{total}")
            current_category = category
            total = price

    if current_category is not None:
        print(f"{current_category}\t{total}")
    _rewind_stream(sys.stdin)
    
    
if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Overwriting CategoryPriceCount.py


In [2]:
! hdfs dfs -rm -r /user/output/ads || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="CategoryPriceCount" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-k1,1 -n" \
    -D mapreduce.map.output.key.field.separator='\t' \
    -files CategoryPriceCount.py \
    -mapper "python3 CategoryPriceCount.py map" \
    -reducer "python3 CategoryPriceCount.py reduce" \
    -input /user/input/ads \
    -output /user/output/ads

Deleted /user/output/ads
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob3770900210737314981.jar tmpDir=null
2025-02-10 06:41:27,927 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-88s08077gbn3tl4n.mdb.yandexcloud.net/10.128.0.14:8032
2025-02-10 06:41:28,091 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-88s08077gbn3tl4n.mdb.yandexcloud.net/10.128.0.14:10200
2025-02-10 06:41:28,125 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-88s08077gbn3tl4n.mdb.yandexcloud.net/10.128.0.14:8032
2025-02-10 06:41:28,125 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-88s08077gbn3tl4n.mdb.yandexcloud.net/10.128.0.14:10200
2025-02-10 06:41:28,340 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1739169572115_0001
2025-02-10 06:41:29,062 INFO mapred.FileInputFormat: Total input files to p

In [3]:
!hdfs dfs -ls /user/output/ads

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2025-02-10 06:43 /user/output/ads/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        964 2025-02-10 06:43 /user/output/ads/part-00000


In [4]:
!hdfs dfs -cat /user/output/ads/part-00000

2	270740247372.0
3	269719275202.0
4	386615276678.0
5	194378853180.0
6	1081387755737.0
8	72750.0
9	368983032949.0
10	778047753029.0
11	286397794715.0
13	360294036962.0
14	1818747309298.0
15	1175981626.0
16	4898855593.0
18	1189957510095.0
19	1225981598150.0
20	1112546355170.0
22	3615683686904.0
23	438290250010.0
25	510975326382.0
26	1516170218902.0
27	752807821253.0
29	103222877542.0
30	956113114474.0
31	4055085405969.0
32	19513368275737.0
33	10701372760148.0
34	6296233036403.0
35	134053168200.0
36	179741332416.0
37	2833544249369.0
38	1911845488400.0
41	8331383550142.0
42	9846683556066.0
43	8988120021083.0
44	93567453953.0
45	1873728809331.0
46	219880267974.0
47	1143968570527.0
48	1040727706890.0
50	5021543119173.0
51	588637293.0
52	862360905.0
53	1305060032868.0
54	13857529612184.0
57	537704430655.0
59	742623523685.0
60	1277750145478.0
250001	3880374977875.0
250004	14254404906260.0
250005	17795315492793.0
250006	8753614085158.0
500001	1799885981114.0


In [10]:
!hdfs dfs -get /user/output/ads/part-00000 part-00000
!hdfs dfs -put -f part-00000 s3a://bspanfilov-lsml/CategoryPriceCount.txt

2025-02-10 06:56:41,260 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-02-10 06:56:41,332 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-02-10 06:56:41,332 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2025-02-10 06:56:43,038 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2025-02-10 06:56:43,038 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2025-02-10 06:56:43,038 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


ссылка на результат: https://storage.yandexcloud.net/bspanfilov-lsml/CategoryPriceCount.txt